In [2]:
import numpy 
import pandas
import nltk
import csv
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##### Los topicos se encuentran juntos en el dataframe, esta función los separa de acuerdo al tema

In [29]:
def separaTopico(topico, df): #Separa todos los encabezados de acuerdo a su tópico
    listTopic = []
    dataArr = numpy.asarray(df)
    for i in range(len(dataArr)):
        if(dataArr[i][1] == topico):
            listTopic.append(dataArr[i][0])
    return listTopic

##### Cuando se tienen las palabras unicas de cada tópico, se eliminan caracteres especiales.

In [79]:
def limpiaTexto(text):
    "List all the word tokens in a text."
    return re.findall('[a-zA-Z0-9\-]{4,254}', text.lower())


##### Cada encabezado corresponde a un documento, esta función separa cada palabra para agregarlas a un solo array

In [19]:
def PalabrasTopico(listTopico): #Devuelve todas las palabras que aparecen en los titulares de las noticias 
    listHeadLines = []
    for i in range(len(listTopico)):
        #aux = nltk.word_tokenize(arrayTopico[i])
        aux = limpiaTexto(listTopico[i])
        for j in range(len(aux)):
            aux2 = aux[j]
            listHeadLines.append(aux2)
            
    return numpy.asarray(listHeadLines)

In [20]:
def Probabilidad(palabrasTopico, palabrasUnicasTopico): #Devuelve un array con la ocurrencias de cada única del tópico
    probabilidad = numpy.zeros(len(palabrasUnicasTopico))
    for i in range(len(palabrasUnicasTopico)):
        probabilidad[i] = numpy.sum(palabrasTopico == palabrasUnicasTopico[i])
        probabilidad[i] = probabilidad[i] /len(palabrasTopico)
        
    return probabilidad

In [21]:
def probTopico(dataTrain, arrayTopic):
    return len(arrayTopic)/len(dataTrain)

##### Abrimos el dataset con pandas

In [22]:
nombres = ['Titular', 'Tópico']
dataTrain = pandas.read_csv('titulares.csv', names=nombres)
dataTrain

,Titular,Tópico
0,Que política o economía conduzcan respuesta al...,covid
1,Oaxaca habilita red de hospitales para covid-19,covid
2,Surgen más brotes de covid-19 en asilo y empre...,covid
3,Suman cuatro mil 477 muertos y 42 mil 595 cont...,covid
4,Revelan que más de mil 400 mujeres han solicit...,covid
...,...,...
1195,"En los Parapanamericanos, México suma ya 42 or...",deportes
1196,El futbolista Jonathan Fabbro es condenado a 1...,deportes
1197,México cosecha 50 medallas en el paraatletismo...,deportes
1198,Debut y gol de Edson Álvarez que da pase al Aj...,deportes


##### Separamos del dataset por tópicos y a cada encabezado lo separamos por palabra de los documentos eliminando caracteres especiales

In [80]:
palabrasCovid = PalabrasTopico(separaTopico('covid', dataTrain))
AlfabetoCovid = numpy.unique(palabrasCovid)
#print(len(palabrasCovid))
#print(len(AlfabetoCovid))


palabrasTecn = PalabrasTopico(separaTopico('tecnologia', dataTrain))
AlfabetoTecn = numpy.unique(palabrasTecn)

palabrasDep = PalabrasTopico(separaTopico('deportes', dataTrain))
AlfabetoDep = numpy.unique(palabrasDep)

Alfabeto = numpy.concatenate((AlfabetoCovid, AlfabetoTecn, AlfabetoDep))
Alfabeto.shape

(3823,)

##### Obtenemos las probabilidades a priori, haciendo cada una de estas equiprobabloe $ P(C) $

In [81]:
PCovid = 1/3
PTecn = 1/3
PDep = 1/3
print("PCovid = ", PCovid, "PTecn = ", PTecn, "PDep = ", PDep)

PCovid =  0.3333333333333333 PTecn =  0.3333333333333333 PDep =  0.3333333333333333


##### Obtenemos la frecuencia de palabras de cada tópico

In [93]:
ProbCovid = Probabilidad(palabrasCovid, AlfabetoCovid)
ProbTecn = Probabilidad(palabrasTecn, AlfabetoTecn)
ProbDep = Probabilidad(palabrasDep, AlfabetoDep)
print(ProbDep)

[0.00035075 0.00035075 0.0038583  ... 0.00035075 0.00070151 0.00035075]


##### Calculamos la probabilidad de cada clase, dado una cadena

In [83]:
def Pword(word, alfabeto):
    P = numpy.sum(alfabeto == word)
    P = P/len(alfabeto)
    return P

$ P(Covid) $

In [90]:
texto = "pandemia"

In [91]:
def Pmax (cadena):
    Proba = numpy.zeros(3)
    
    for i in range(len(AlfabetoCovid)):
        if(AlfabetoCovid[i] == texto):
            Proba[0] = PCovid * ProbCovid[i]
            
    for i in range(len(AlfabetoTecn)):
        if(AlfabetoTecn[i] == texto):
            Proba[1] = PTecn * ProbTecn[i]
            
    for i in range(len(AlfabetoDep)):
        if(AlfabetoDep[i] == texto):
            Proba[2] = PTecn * ProbDep[i]
            
    return Proba

In [92]:
result = Pmax(texto)
print(result)
masRes = max(result)
masRes

[0.00426829 0.         0.00035075]


0.004268292682926829